# Azure Load Testing: Automation

Azure Load Testing ist ein Dienst von Azure, mit dem man schnell und einfach hohe Lasten aus der Azure Cloud heraus mit Hilfe von Apache JMeter generieren kann:

### Azure CLI

Die Steuerung erfolgt komplett über die Azure CLI. Alle Schritte können vermutlich auch mit Terraform gelöst werden oder per Hand umgesetzt werden. Dafür installieren wir den CLI client über `scoop` und loggen uns dann ein:

In [ ]:
!scoop install azure-cli

In [ ]:
!az config set extension.use_dynamic_install=yes_without_prompt
!az login
!start https://portal.azure.com/

Jetzt ist es möglich die Azure CLI Befehle direkt aus der Kommandozeile zu verwenden.

### Azure Web App

Warum  machen wir das eigentlich? Im Endeffekt geht es darum, eine Applikation unter Last zu setzen und dann auszuwerten, wie sie sich verhält. Das bedeutet, dass wir zunächst eine App brauchen, die wir abtesten können. Dafür können wir eine ganz simple python app in Azure erstellen.
Die App, die wir hier benutzen, besitzt 1 URL-Argument `sleep`, was in der App eine entsprechende Wartezeit auslöst. Die App berechnet nichts, aber über die `sleep`-Funktionalität kann man eine entsprechende Antwortzeit simulieren.
Der Code der hier verwendeten Web-App (Python) befindet sich in einem Extra Github-Repository (https://github.com/besessener/SimplePythonAppDemo). Es handelt sich um eine sehr einfache Python Anwendung, mit der ich Last über einen `sleep` simulieren kann.

In [ ]:
!az appservice plan create --name QyteraAppPlan --resource-group Qload --sku FREE --is-linux --location germanywestcentral
!az webapp create --resource-group Qload -p QyteraAppPlan -n QyteraWebApp --runtime "python:3.10" --deployment-source-url https://github.com/besessener/SimplePythonAppDemo.git

### Azure Load Testing Aufsetzen

Wir benutzen das Azure CLI jetzt, um eine Load Testing Ressource anzulegen. Dafür muss bereits eine `Resource Group` existieren: in diesem Beispiel heißt sie `Qload`. Zudem muss man der Load Testing Resource einen Namen geben: `QyteraLoadTesting`:

In [ ]:
!az load create --name QyteraLoadTesting --resource-group Qload --location germanywestcentral

### Einen Test Hinzufügen

Dann fügen wir einen Test hinzu. Der Test beinhaltet direkt das JMX Script und eine CSV Datei. Dafür ist es hilfreich JMeter einmal zu öffnen und das Script anzusehen.

In [ ]:
!call jmeter -t ../jmeter/jmeter-script.jmx

Jetzt laden wir das Script hoch und erstellen damit den Test (`<your APP ID>` ersetzen).

In [ ]:
!az load test create --load-test-resource QyteraLoadTesting --test-id testrunner --display-name "My CLI Load Test" --description "Created using Az CLI" --test-plan "../jmeter/jmeter-script.jmx" --engine-instances 1 --resource-group Qload
!az load test app-component add --test-id testrunner --load-test-resource QyteraLoadTesting --resource-group Qload --app-component-name QyteraWebApp --app-component-type  microsoft.web/sites --app-component-id "<your APP ID>" --app-component-kind web
!az load test file upload --test-id testrunner --load-test-resource QyteraLoadTesting --resource-group Qload --path "../jmeter/jmeter-data.csv" --file-type ADDITIONAL_ARTIFACTS

### Test Durchführen

In [ ]:
!call az load test-run create --load-test-resource QyteraLoadTesting --resource-group Qload --test-id testrunner --test-run-id first --display-name first --description "First Test run from CLI" --env USER_COUNT=1

### Auswertung

Die Auswertung der Testläufe kann man nun wie gewohnt über das CLI machen:

In [ ]:
import subprocess
import json
import matplotlib.pyplot as plt

# 1. Execute the az command and get the output
cmd = "az load test-run metrics list --load-test-resource QyteraLoadTesting --resource-group Qload --test-run-id first --metric-namespace LoadTestRunMetrics --interval PT5S"

result = subprocess.run(cmd.split(), capture_output=True, text=True, shell=True)
data = json.loads(result.stdout)

# 2. Extract the timestamp and value from the JSON response for plotting
timestamps = [entry["timestamp"] for entry in data["ResponseTime"][0]["data"]]
values = [entry["value"] for entry in data["ResponseTime"][0]["data"]]

# 3. Plot the data using matplotlib
plt.figure(figsize=(10, 6))
plt.plot(timestamps, values, '-o', label='Response Time')
plt.xticks(rotation=45)
plt.title('Response Time vs. Timestamp')
plt.xlabel('Timestamp')
plt.ylabel('Response Time (ms)')
plt.tight_layout()
plt.legend()
plt.grid(True)
plt.show()


oder auch per GUI.

## Aufräumen

Um sicher zu gehen, dass keine zusätzlichen Kosten anfallen, sollte man am Ende auch alle unbenutzten Resourcen einfach wieder löschen:

In [ ]:
!az webapp delete --resource-group Qload --name QyteraWebApp
!az load test delete --load-test-resource QyteraLoadTesting --resource-group Qload --test-id testrunner --yes
!az load delete --name QyteraLoadTesting --resource-group Qload --yes